In [2]:
%load_ext autoreload
%autoreload 2

In [4]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))


In [6]:
import pandas as pd
from src.config import TRANSFORMED_DATA_DIR

# Load your Citi Bike dataset
df = pd.read_parquet(TRANSFORMED_DATA_DIR / "citibike_tabular_data_28d.parquet")
df.head()


,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,target,pickup_hour,pickup_location_id
0,1,0,0,0,0,0,0,0,0,0,...,0,0,5,12,15,12,4,6,2023-01-25 11:00:00,5329.03
1,0,0,0,0,0,0,0,0,0,0,...,0,1,5,12,21,6,8,8,2023-01-26 11:00:00,5329.03
2,0,0,0,0,0,0,0,0,0,0,...,0,1,1,10,12,6,8,3,2023-01-27 11:00:00,5329.03
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,6,5,10,23,2023-01-28 11:00:00,5329.03
4,15,12,10,24,23,14,12,6,5,5,...,0,0,0,1,2,9,6,14,2023-01-29 11:00:00,5329.03


In [8]:
from datetime import datetime
from src.data_utils import split_time_series_data

X_train, y_train, X_test, y_test = split_time_series_data(
    df,
    cutoff_date=datetime(2023, 9, 1, 0, 0, 0),  # Use the right cutoff
    target_column="target"
)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


(657, 674)
(657,)
(366, 674)
(366,)


In [10]:
X_train

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_hour,pickup_location_id
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,5,12,15,12,4,2023-01-25 11:00:00,5329.03
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,5,12,21,6,8,2023-01-26 11:00:00,5329.03
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,10,12,6,8,2023-01-27 11:00:00,5329.03
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,6,5,10,2023-01-28 11:00:00,5329.03
4,15,12,10,24,23,14,12,6,5,5,...,0,0,0,0,1,2,9,6,2023-01-29 11:00:00,5329.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
896,23,22,20,50,46,28,38,37,30,17,...,1,0,2,2,0,2,14,8,2023-08-27 11:00:00,6948.10
897,17,20,19,12,29,27,43,48,27,13,...,0,1,2,3,5,15,11,9,2023-08-28 11:00:00,6948.10
898,14,12,9,11,12,12,26,23,26,12,...,2,2,2,3,4,23,15,11,2023-08-29 11:00:00,6948.10
899,16,13,13,9,20,17,31,20,15,6,...,1,2,2,9,3,15,0,7,2023-08-30 11:00:00,6948.10


In [12]:
# Select only past ride features
past_ride_columns = [c for c in X_train.columns if c.startswith("rides_")]
X_train_only_numeric = X_train[past_ride_columns]
X_test_only_numeric = X_test[past_ride_columns]


In [14]:
import xgboost as xgb

model = xgb.XGBRegressor(max_depth=10)
model.fit(X_train_only_numeric, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [16]:
from sklearn.metrics import mean_absolute_error

predictions = model.predict(X_test_only_numeric)
test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae:.4f}")


5.9308


In [18]:
from src.experiment_utils import set_mlflow_tracking, log_model_to_mlflow
from dotenv import load_dotenv
import os

load_dotenv()
mlflow = set_mlflow_tracking()

log_model_to_mlflow(model, X_test_only_numeric, "XGBoost-Citibike", "mean_absolute_error", score=test_mae)


INFO:src.experiment_utils:MLflow tracking URI and credentials set.
2025/05/09 21:28:09 INFO mlflow.tracking.fluent: Experiment with name 'XGBoost-Citibike' does not exist. Creating a new experiment.
INFO:src.experiment_utils:Experiment set to: XGBoost-Citibike
INFO:src.experiment_utils:Logged mean_absolute_error: 5.930795101520142
/opt/anaconda3/lib/python3.12/site-packages/mlflow/types/utils.py:435: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest

🏃 View run fearless-auk-754 at: https://dagshub.com/sahilsubhasbhaivachhani/final_project.mlflow/#/experiments/3/runs/7488dad7cc024952877bd74ca2aa082f
🧪 View experiment at: https://dagshub.com/sahilsubhasbhaivachhani/final_project.mlflow/#/experiments/3
